In [ ]:
import pandas as pd
from tqdm import tqdm
import json
from google.oauth2 import service_account

In [ ]:
registration_file_str = r'C:\Users\thewh\Downloads\registration-timeseries.csv'
turnout_file_str = r'C:\Users\thewh\Documents\Constellation Political\Research\After Action Reports\co-turnout-rates.csv'

non_flt_col_lst = [
    'Date',
    'District',
    'District_Type',
    'Year'
    'RTLA'
]

# BQ Variables
bq_project_name = os.environ.get('BQ_PROJECT_ID')
bq_project_location = 'us-west1'
bq_dataset_name = 'co_voterfile'
bq_timeseries_table_name = 'registration-timeseries'
bq_timeseries_table_id = f'{bq_project_name}.{bq_dataset_name}.{bq_timeseries_table_name}'

# Establish BigQuery credentials
bq_account_creds = json.loads(os.environ.get('BQ_ACCOUNT_CREDS'))
bq_credentials = service_account.Credentials.from_service_account_info(bq_account_creds)


In [ ]:
registration_df = pd.read_csv(registration_file_str)
turnout_df = pd.read_csv(turnout_file_str)

In [ ]:
for col in tqdm(list(registration_df)):
    if col not in non_flt_col_lst:
        registration_df[col] = pd.to_numeric(registration_df[col], errors='coerce')
        registration_df[col] = registration_df[col].astype('Int64')
registration_df['RTLA'] = registration_df['RTLA'].astype('float64')

for col in tqdm(list(turnout_df)):
    if col not in non_flt_col_lst:
        turnout_df[col] = pd.to_numeric(turnout_df[col], errors='coerce')
        turnout_df[col] = turnout_df[col].astype('float64')

In [ ]:
registration_df['Date'] = pd.to_datetime(registration_df['Date'])

In [ ]:
for i in tqdm(registration_df.index):
    row_year_int = registration_df.loc[i, 'Date'].year
    if row_year_int < 2010:
        turnout_year_int = 2010
    else:
        turnout_year_int = (row_year_int) + (row_year_int % 2)
    
    year_int, district_str, rep_flt, dem_flt, uaf_flt, oth_flt, tot_flt = turnout_df[(turnout_df['Year'] == turnout_year_int) & (turnout_df['District'] == registration_df.loc[i, 'District'])].iloc[0]

    cast_tot_flt = (registration_df.loc[i, 'REP_TOT'] * rep_flt) + (registration_df.loc[i, 'DEM_TOT'] * dem_flt) + (registration_df.loc[i, 'UAF_TOT'] * uaf_flt) + (registration_df.loc[i, 'OTH_TOT'] * oth_flt)
    win_tot_flt = cast_tot_flt * 0.51
    uaf_votes_flt = win_tot_flt - (registration_df.loc[i, 'REP_TOT'] * rep_flt)
    rtla_flt = uaf_votes_flt / ((registration_df.loc[i, 'UAF_TOT'] * uaf_flt) + (registration_df.loc[i, 'OTH_TOT'] * oth_flt))

    registration_df.loc[i, 'RTLA'] = rtla_flt

    

In [ ]:
def create_bq_schema(df: pd.DataFrame) -> list:
    schema_list = []
    for column in list(df):
        if 'date' in column.lower():
            sql_type = 'DATE'
        elif column in ['RTLA']:
            sql_type = 'FLOAT'
        elif column in ['District', 'District_Type']:
            sql_type = 'STRING'
        else:
            sql_type = 'INT64'
                
        if column in ['District', 'Date']:
            sql_mode = 'REQUIRED'
        else:
            sql_mode = 'NULLABLE'
            
        schema_list.append({'name':  column, 'type': sql_type, 'mode': sql_mode})
    
    return schema_list

In [ ]:
bq_schema_lst = create_bq_schema(registration_df)

In [ ]:
registration_df.to_gbq(destination_table=bq_timeseries_table_id, project_id=bq_project_name, if_exists='replace', table_schema=bq_schema_lst, credentials=bq_credentials)
